<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       Banking Customer Churn Analysis using already deployed models
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<p style ='font-size:20px;font-family:Arial'><b>Introduction</b></p>

<p style ='font-size:16px;font-family:Arial'>This Notebook is a part of the Banking Customer Churn use case showing the AutoChurn functionality.</p>

<p style='font-size:16px;font-family:Arial'>In this Notebook we will use the models which we created using the AutoChurn function in the <b><code>Bank_Customer_Churn_Prediction_Deploy_Models.ipynb</code></b> notebook and saved those models in database for future use.</p>
 
<p style='font-size:16px;font-family:Arial'>This notebook will load the deployed models and use these saved models to do predictions on the data.</p>

<hr style="height:1px;border:none;">
<p style = 'font-size:18px;font-family:Arial'><b>Importing libraries need for execution. </b>

<p style = 'font-size:16px;font-family:Arial'>Here, we import the required libraries, set environment variables and environment paths (if required).</p>

In [ ]:
import getpass
from teradataml import (AutoML, 
                        AutoChurn,
                        create_context, 
                        remove_context,
                        DataFrame,
                        TrainTestSplit,
                        execute_sql,
                        in_schema)

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>1. Initiate a connection to Vantage</b>
<p style = 'font-size:16px;font-family:Arial'>You will be prompted to provide the password. Enter your password, press the Enter key, and then use the down arrow to go to the next cell.</p>

In [ ]:
%run -i ../startup.ipynb
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)

In [ ]:
%%capture
execute_sql('''SET query_band='DEMO=BankingChurn_AutoChurn_Load.ipynb;' UPDATE FOR SESSION; ''')

<p style = 'font-size:16px;font-family:Arial'>Begin running steps with Shift + Enter keys. </p>

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>2. Loading Deployed Models - 'top_10_models' </b>

<b style = 'font-size:18px;font-family:Arial'>2.1. Loading Model </b>

In [ ]:
# Creating AutoML object

aml=AutoChurn()

In [ ]:
# Loading models

models_1 = aml.load('top_10_models')

In [ ]:
# Display loaded models

models_1

In [ ]:
### Loading Dataset for Prediction

df = DataFrame(in_schema("DEMO_BankChurn", "customer_churn"))

In [ ]:
# Display data

df

In [ ]:
df.shape

<hr style="height:1px;border:none;">
<b style = 'font-size:18px;font-family:Arial'>2.2. Generating Prediction & Performance Metrics</b>

In [ ]:
# Generate prediction using some data rows and model rank

prediction = aml.predict(df.iloc[:80], rank=1)

In [ ]:
prediction

In [ ]:
# Generate performance metrics

performance_metric = aml.evaluate(df.iloc[:80], rank=1)

In [ ]:
performance_metric

In [ ]:
# Generate prediction using data and model rank

prediction = aml.predict(df.iloc[:80], rank=7)

In [ ]:
prediction

In [ ]:
# Generate performance metrics

performance_metric = aml.evaluate(df.iloc[:80], rank=7)

In [ ]:
performance_metric

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>3. Loading Deployed Models - 'mixed_models' </b>

<b style = 'font-size:18px;font-family:Arial'>3.1. Loading Model </b>

In [ ]:
churn = AutoChurn()

In [ ]:
# Loading models

models_2 = churn.load('mixed_models')

In [ ]:
models_2

<hr style="height:1px;border:none;">
<b style = 'font-size:18px;font-family:Arial'>3.2. Generating Prediction & Performance Metrics</b>

In [ ]:
# Generate prediction using data and model rank

prediction = churn.predict(df.iloc[:80], rank=4)

In [ ]:
prediction

In [ ]:
# Generate performance metrics

performance_metric = churn.evaluate(df.iloc[:80], rank=4)

In [ ]:
performance_metric

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>4. Loading Deployed Models - 'range_models' </b>

<b style = 'font-size:18px;font-family:Arial'>4.1.</span> Loading Model</b>

In [ ]:
# Creating another AutoML object

obj=AutoChurn()

In [ ]:
# Loading models

models_3 = obj.load('range_models')

In [ ]:
models_3

<hr style="height:1px;border:none;">
<b style = 'font-size:18px;font-family:Arial'>4.2.</span> Generating Prediction & Performance Metrics</b>

In [ ]:
# Generate prediction using data and model rank

prediction = obj.predict(df.iloc[:80], rank=1)

In [ ]:
prediction

In [ ]:
# Generate performance metrics

performance_metric = obj.evaluate(df.iloc[:80], rank=1)

In [ ]:
performance_metric

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>5. Cleanup</b>

<div class="alert alert-block alert-warning">
<p style = 'font-size:16px;font-family:Arial'><b>Note: </b>The cleanup process will drop all tables and also models created as a part of AutoChurn process. Make sure the tables will not be needed, to avoid rework, before executing the cleanup process. </p>
</div>    

<p style = 'font-size:18px;font-family:Arial'><b>Work Tables</b></p>
<p style = 'font-size:16px;font-family:Arial'>We need to clean up our work tables to prevent errors next time.</p>

In [ ]:
tables = ['top_10_models', 'mixed_models', 'range_models']

# Loop through the list of tables and execute the drop table command for each table
for table in tables:
    try:
        db_drop_table(table_name = table)
    except:
        pass

<p style = 'font-size:18px;font-family:Arial'> <b>Databases and Tables </b></p>
<p style = 'font-size:16px;font-family:Arial'>We will use the following code to clean up tables and databases created for this demonstration.</p>

In [ ]:
# %run -i ../run_procedure.py "call remove_data('DEMO_BankChurn_cloud');"        # Takes 5 seconds
%run -i ../run_procedure.py "call remove_data('DEMO_BankChurn_local');"        # Takes 5 seconds

In [ ]:
remove_context()

<footer style="padding-bottom:35px; border-bottom:3px solid #91A0Ab">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2026. All Rights Reserved
        </div>
    </div>
</footer>